In [1]:
from src.utils.file_functions import switch_suffix_file
from src.utils.find_buried_filetypes import single_path_find_buried_files

from src.dict_parameters_data import parameters_filepath_to_dict

import struct
import numpy as np
from os.path import getsize, join

In [2]:
filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/X_TEST/scanfiles0002/field_0001'

In [18]:

import struct
import numpy as np

from src.utils.file_functions import switch_suffix_file
from src.dict_parameters_data import parameters_filepath_to_dict

from os.path import getsize

from src.filetype_key_lists import field_key_list, field_column_keys


def read_field_file(filepath, choose_time = 'last', named_field_col = 'all'):
    parameter_filepath = switch_suffix_file(filepath, 'parameters')
    param_dict = parameters_filepath_to_dict(parameter_filepath)

    field_dict = {'filepath': filepath,
                  'key_list': field_key_list,
                  'time': None, 
                  'phi':  [],
                  'apar': [],
                  'bpar': []}

    with open(filepath, 'rb') as file:
        nx, ny, nz, n_fields, precision, endianness = (
            param_dict['nx0'],
            param_dict['nky0'],
            param_dict['nz0'],
            param_dict['n_fields'],
            param_dict['PRECISION'],
            param_dict['ENDIANNESS']
        )

        intsize = 4
        realsize = 8 if precision == 'DOUBLE' else 4
        complexsize = 2 * realsize
        entrysize = nx * ny * nz * complexsize
        leapfld = n_fields * (entrysize + 2 * intsize)

        complex_dtype = np.dtype(np.complex64) if precision == 'SINGLE' else np.dtype(np.complex128)

        format_string = '>' if endianness == 'BIG' else '='
        format_string += 'ifi' if precision == 'SINGLE' else 'idi'
        timeentry = struct.Struct(format_string)
        timeentry_size = timeentry.size


        all_time_values = []
        for _ in range(int(getsize(filepath) / (leapfld + timeentry_size))):
            time = float(timeentry.unpack(file.read(timeentry_size))[1])
            file.seek(leapfld, 1)
            all_time_values.append(time)

        
        if choose_time == 'last':
            time_index_list = [all_time_values.index(max(all_time_values))]
        elif choose_time == 'first':
            time_index_list = [all_time_values.index(min(all_time_values))]
        elif isinstance(choose_time, float) or isinstance(choose_time, int):
            time_index_list = [all_time_values.index(choose_time)]
        elif isinstance(choose_time, list) and len(choose_time)==2:
            start_time, end_time = choose_time
            if start_time > end_time:
                raise ValueError (f'Ensure initial time value ({start_time}) is less than final time value ({end_time})')
            time_index_list = [time_ind for time_ind, time in enumerate(all_time_values) if start_time <= time <= end_time]
        else:
            raise ValueError
        
        field_dict['time'] = [all_time_values[time_ind] for time_ind in time_index_list]
            

        if named_field_col == 'all':
            field_names = field_column_keys
        else:
            field_names = [named_field_col] if isinstance(named_field_col, str) else named_field_col


        for time_index in time_index_list:

            for field_name in field_names:
                ind = field_column_keys.index(field_name)
                offset = timeentry_size + time_index * (timeentry_size + leapfld) + ind * (entrysize + 2 * intsize) + intsize
                file.seek(offset)

                field_dict[field_name].append(np.fromfile(file, count=nx * ny * nz, dtype=complex_dtype).reshape(nz, ny, nx))
        
    return field_dict


In [28]:
desired_time = 0.5360000000010587
desired_time = [0.002,.01]

filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/X_TEST/scanfiles0002/field_0003'

field_dict = read_field_file(filepath, choose_time=desired_time, named_field_col='phi')

field_dict = read_field_file(filepath, choose_time=desired_time)

# print(field_dict.keys())

for key in field_dict.keys():
    print(key, len(field_dict[key]))

    if key == 'phi':
        for i in range(len(field_dict[key])):
            pass
            # print(i, field_dict[key][i])



filepath 71
key_list 4
time 12
phi 12
apar 12
bpar 12


In [5]:
#!/usr/bin/env python
import struct
from os.path import getsize, join
import numpy as np


class fieldfile(object):
    # class constructor
    def __init__(self, file, pars):
        self.pars = pars
        self.file = file
        self.set_gridcounts()
        self.set_sizes()
        self.set_datatypes()
        self.te, self.tesize = self.TimeEntry()
        self.define_arrays()
        self.redirect(self.file)

    # call this routine to read from a new field file
    def redirect(self, file):
        self.file = file
        try:
            self.f.close()
        except (AttributeError, OSError):
            pass
        self.f = open(file, 'rb')
        self.tfld = []
        self.get_timearray()
        self.reset_tinds()

    # set resolution
    def set_gridcounts(self):
        self.nx = int(self.pars['nx0'])
        self.ny = int(self.pars['nky0'])
        self.nz = int(self.pars['nz0'])

    # entry sizes
    def set_sizes(self):
        self.nfields = int(self.pars['n_fields'])
        self.intsize = 4
        realsize = 4 + (self.pars['PRECISION'] == 'DOUBLE')*4
        complexsize = 2*realsize
        self.entrysize = self.nx*self.ny*self.nz*complexsize
        # jumps in bytes in field/mom files
        self.leapfld = self.nfields*(self.entrysize + 2*self.intsize)

    # real and complex datatypes according to endianness
    def set_datatypes(self):
        if self.pars['PRECISION'] == 'DOUBLE':
            nprt = np.dtype(np.float64)
            npct = np.dtype(np.complex128)
        else:
            nprt = np.dtype(np.float32)
            npct = np.dtype(np.complex64)
        try:
            self.bigendian = self.pars['ENDIANNESS'] == 'BIG'
        except KeyError:
            self.bigendian = False
        if self.bigendian:
            self.nprt = (nprt).newbyteorder()
            self.npct = (npct).newbyteorder()
        else:
            self.nprt = nprt
            self.npct = npct

    def define_arrays(self):
        self.phi3d = np.empty((self.nz, self.ny, self.nx), dtype=self.npct)
        self.apar3d = np.empty((self.nz, self.ny, self.nx), dtype=self.npct)
        self.bpar3d = np.empty((self.nz, self.ny, self.nx), dtype=self.npct)

    def get_timearray(self):
        # get time arrays for field file
        for i in range(int(getsize(self.file)/(self.leapfld + self.tesize))):
            self.tfld.append(float(self.te.unpack(self.f.read(self.tesize))[1]))
            self.f.seek(self.leapfld, 1)

    def get_minmaxtime(self):
        if not self.tfld:
            self.get_timearray()
        return self.tfld[0], self.tfld[-1]

    # defines the struct for a time entry in field
    def TimeEntry(self):
        if self.bigendian:
            if self.pars['PRECISION'] == 'SINGLE':
                timeentry = struct.Struct('>ifi')
            else:
                timeentry = struct.Struct('>idi')
        else:
            if self.pars['PRECISION'] == 'SINGLE':
                timeentry = struct.Struct('=ifi')
            else:
                timeentry = struct.Struct('=idi')
        return timeentry, timeentry.size

    # calculate offset in field file for a given timestep and variable
    def offset(self, var):
        if var in [i for i in range(self.nfields)]:
            return self.tesize + self.tind*(self.tesize+self.leapfld) + var*(
                   self.entrysize + 2*self.intsize) + self.intsize

    # returns field for given timestep
    def readvar(self, var):
        self.f.seek(self.offset(var))
        var3d = np.fromfile(self.f, count=self.nx*self.ny*self.nz, dtype=self.npct).reshape(self.nz,
                                                                                            self.ny,
                                                                                            self.nx)
        return var3d

    # return time index for given time, if it is present
    # otherwise, an exception is raised
    def get_tind(self):
        return self.tfld.index(self.time)

    # set current timestep
    def set_time(self, time):
        self.time = time
        self.tind = self.get_tind()

    # reset time indices
    def reset_tinds(self):
        self.tind = 0
        self.ftind = [-1]*self.nfields

    # return phi. this will only read from file if necessary
    def phi(self):
        if self.ftind[0] == self.tind:
            pass
        else:
            self.ftind[0] = self.tind
            self.phi3d = self.readvar(0)
        return self.phi3d

    # same for apar
    def apar(self):
        if self.nfields > 1:
            if self.ftind[1] == self.tind:
                pass
            else:
                self.ftind[1] = self.tind
                self.apar3d = self.readvar(1)
            return self.apar3d

    # same for bpar
    def bpar(self):
        if self.nfields > 2:
            if self.ftind[2] == self.tind:
                pass
            else:
                self.ftind[2] = self.tind
                self.bpar3d = self.readvar(2)
            return self.bpar3d


In [6]:
# input_filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/X_TEST'
input_filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/129015/r_0.936008_OM_top/MTM_limit'

field_path_list = single_path_find_buried_files(input_filepath, 'field')
print(len(field_path_list))

504


In [7]:
for filepath in field_path_list:
    parameter_filepath = switch_suffix_file(filepath, 'parameters')
    param_dict = parameters_filepath_to_dict(parameter_filepath)

    field = fieldfile(filepath, param_dict)

    desired_time = 0

    field.set_time(desired_time)
    # print(field.get_tind())

    phi_data_class = np.array(field.phi())
    apar_data_class = np.array(field.apar())
    bpar_data_class = np.array(field.bpar())

    print(phi_data_class.shape, apar_data_class.shape, bpar_data_class.shape)
    # print(phi_data_class.shape)

(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)


(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128, 1, 7)
(128, 1, 7) (128, 1, 7) (128

In [8]:
for filepath in field_path_list:
    desired_time = 0
    field_dict = read_field_file(filepath, desired_time, named_field_col='all')
    print(field_dict['phi'][0].shape, field_dict['apar'][0].shape, field_dict['bpar'][0].shape, len(field_dict['time']))
    # print(field_dict['phi'][0].shape)

(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1


(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) (128, 1, 7) (128, 1, 7) 1
(128, 1, 7) 

In [9]:
parameter_filepath = switch_suffix_file(filepath, 'parameters')
param_dict = parameters_filepath_to_dict(parameter_filepath)

field = fieldfile(filepath, param_dict)

desired_time = 0

field.set_time(desired_time)
# print(field.get_tind())

phi_data_class = np.array(field.phi())
apar_data_class = np.array(field.apar())
bpar_data_class = np.array(field.bpar())










# # print(phi_data_class[0])
# # print(phi_data_class[-1])

# # print()

# # print(apar_data_class[0])
# # print(apar_data_class[-1])

# # print()

# # print(bpar_data_class[0])
# # print(bpar_data_class[-1])




In [10]:
filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/X_TEST/scanfiles0002/field_0001'

# parameter_filepath = switch_suffix_file(filepath, 'parameters')
# param_dict = parameters_filepath_to_dict(parameter_filepath)
# print(parameter_filepath)


# field = fieldfile(filepath, param_dict)
# field_dict = read_field_file(filepath, 0)


all_check = []

desired_time = 0


for filepath in field_path_list:

    parameter_filepath = switch_suffix_file(filepath, 'parameters')
    param_dict = parameters_filepath_to_dict(parameter_filepath)

    field = fieldfile(filepath, param_dict)

    field.set_time(desired_time)
    # print(field.get_tind())

    phi_data_class = np.array(field.phi())
    apar_data_class = np.array(field.apar())
    bpar_data_class = np.array(field.bpar())

    # print(phi_data_class.shape, apar_data_class.shape, bpar_data_class.shape)


    field_dict = read_field_file(filepath, desired_time)
    # print(field_dict['phi'][0].shape, field_dict['apar'][0].shape, field_dict['bpar'][0].shape, len(field_dict['time']))

    phi_data = field_dict['phi'][0]
    apar_data = field_dict['apar'][0]
    bpar_data = field_dict['bpar'][0]
    

    phi_check = phi_data == phi_data_class
    phi_all_true = all(all(row) for matrix in phi_check for row in matrix)
    # print(phi_all_true)

    apar_check = apar_data == apar_data_class
    apar_all_true = all(all(row) for matrix in apar_check for row in matrix)
    # print(apar_all_true)

    bpar_check = bpar_data == bpar_data_class
    bpar_all_true = all(all(row) for matrix in bpar_check for row in matrix)
    # print(bpar_all_true)

    triple_true_matrix = [phi_all_true, apar_all_true, bpar_all_true]

    triple_check = all(triple_true_matrix)

    print(triple_check)

    all_check.append(triple_check)

print(all(all_check))





True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [11]:
filepath = '/pscratch/sd/j/joeschm/NSXTU_discharges/X_TEST/scanfiles0002/field_0001'
desired_time = 38.210900000105646
phi_data, apar_data, bpar_data, time_values = read_field_file(filepath, desired_time)

# phi_check = phi_data == phi_data_class
# phi_all_true = all(all(row) for matrix in phi_check for row in matrix)
# print(phi_all_true)

# apar_check = apar_data == apar_data_class
# apar_all_true = all(all(row) for matrix in apar_check for row in matrix)
# print(apar_all_true)

# bpar_check = bpar_data == bpar_data_class
# bpar_all_true = all(all(row) for matrix in bpar_check for row in matrix)
# print(bpar_all_true)



# print()

# print(phi_data[0])
# print(phi_data[-1])

# print()

# print(apar_data[0])
# print(apar_data[-1])

# print()

# print(bpar_data[0])
# print(bpar_data[-1])



In [12]:
print(phi_data.shape)

AttributeError: 'str' object has no attribute 'shape'